<a href="https://colab.research.google.com/github/pranshurastogi29/News-summarization-Topic-prediction/blob/main/News_summarization_%26_Topic_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install colorama
!pip install transformers
!pip install -q streamlit

     |████████████████████████████████| 3.4 MB 8.5 MB/s 
     |████████████████████████████████| 895 kB 43.8 MB/s 
     |████████████████████████████████| 3.3 MB 47.0 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 596 kB 48.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 9.3 MB 7.2 MB/s 
     |████████████████████████████████| 4.3 MB 43.8 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 164 kB 69.3 MB/s 
     |████████████████████████████████| 180 kB 69.4 MB/s 
     |████████████████████████████████| 111 kB 56.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 127 kB 55.9 MB/s 
     |████████████████████████████████| 792 kB 46.6 MB/s 
     |████████████████████████████████| 374 kB 

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2022-01-21 12:03:01--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  15.8MB/s    in 0.8s    

2022-01-21 12:03:02 (15.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
import torch
torch.__version__

'1.10.0+cu111'

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
%cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d pranshu29/plain-roberta-base-train
!unzip plain-roberta-base-train.zip -d roberta_train

 96% 280M/291M [00:02<00:00, 134MB/s]
100% 291M/291M [00:02<00:00, 127MB/s]
Archive:  plain-roberta-base-train.zip
  inflating: roberta_train/last-checkpoint.bin  
  inflating: roberta_train/out.csv   


In [ ]:
%%writefile app.py
import streamlit as st
from transformers import pipeline
import time
import pandas as pd
import numpy as np
import joblib
from transformers import AutoModel, AutoTokenizer, AutoConfig
import torch
import torch.nn as nn
import pickle 

class ToxicSimpleNNModel(nn.Module):

    def __init__(self, path):
        super(ToxicSimpleNNModel, self).__init__()
        self.backbone = AutoModel.from_pretrained(path)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(in_features=self.backbone.pooler.dense.out_features*2,out_features=8)
        
    def forward(self, input_ids, attention_masks):
        seq_x, _= self.backbone(input_ids=input_ids, attention_mask=attention_masks, return_dict=False)
        apool = torch.mean(seq_x, 1)
        mpool, _ = torch.max(seq_x, 1)
        x = torch.cat((apool, mpool), 1)
        x = self.dropout(x)
        return self.linear(x)

def load_topic_model(base_path, model_path):
  net = ToxicSimpleNNModel(base_path)
  net.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
  return net

@st.cache(allow_output_mutation=True,suppress_st_warning=True)
def Topic_generation_load(base_path, model_path, tokenizer_path):
    print('loading topic_model')
    with open(tokenizer_path , 'rb') as f: 
      tokenizer = pickle.load(f)
    model = load_topic_model(base_path, model_path)
    return model , tokenizer

@st.cache(allow_output_mutation=True,suppress_st_warning=True)
def load_summarization_model():
    print('loading summarization model')
    summarization_pipe = pipeline('summarization')
    print('sentiment model loading')
    return summarization_pipe

def get_summarization(text, summarization, max_lenght):
  return summarization(text, max_length=max_lenght)[0]['summary_text']


def predict_topic(text, tokenizer):
  encoded = tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=512, 
            pad_to_max_length=True
        )
  tokens = torch.tensor(encoded['input_ids']).unsqueeze(0)
  attention_masks = torch.tensor(encoded['attention_mask']).unsqueeze(0)
  outputs = model(tokens, attention_masks)
  topics = nn.functional.sigmoid(outputs).detach().numpy()
  _ , topics = torch.topk(torch.tensor(topics), dim = 1, k = 3)
  topics = np.array(topics)
  top_dic = {'0':'business','1':'elections','2':'entertainment',
             '3':'news','4':'opinion','5':'sci-tech','6':'society',
             '7':'sport'}
  l = []
  for i in topics[0]:
    l.append(top_dic[str(i)])
  return l

summarization_pipe = load_summarization_model()
model , tokenizer = Topic_generation_load('tiny-bert' ,'model.bin', 'tokenizer.obj')

st.title('News Summary Generation and Topic Prediction')

st.markdown('Here you can enter the News in first text box and can get news summary around the subject')

text = st.text_input('Enter News here:',key=0)

if st.checkbox('Start Generate Summary'):
    st.write('uncheck the box if you are done')
    option = st.sidebar.selectbox(label='Max_Lenght',options=['20','40','50'])
    summary = get_summarization(text, summarization_pipe, int(option))
    st.write('Final summary ' + summary)
else: pass

st.markdown('Once done you can get the top Topics the news relate too')

if st.button('Predict Topics'):
  l = predict_topic(text, tokenizer)
  st.markdown('Top topics related are '+' , '.join(l))
else: pass


Overwriting app.py


In [ ]:
!./ngrok authtoken 21yJFX87jVygzsrMEb2rePp0j1G_K3UkVeGWRtiBDRmuGFRJ

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: http://814d-34-83-152-139.ngrok.io


In [ ]:
!streamlit run "app.py"


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.83.152.139:8501

loading summarization model
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)
sentiment model loading
loading topic_model
Some weights of the model checkpoint at /content/drive/MyDrive/tiny-bert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- 